# Implementing AdaBoost Classifier from scratch
### Álvaro Corrales Cano
### April 2021 - work in progress

In [1]:
# Imports
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from AdaBoost import *
from sklearn.ensemble import AdaBoostClassifier


## Synthetic dataset


In [7]:
# Generate classification dataset
X, y = make_classification(n_samples= 1000, n_features = 20, random_state = 42)
y = y * 2 - 1       # Original AdaBoost uses {1, -1} as class labels

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Custom implementation

In [8]:
# Fit model
ab = AdaBoost()
ab.fit(X_train, y_train, M = 400)

# Predict on test set
y_pred = ab.predict(X_test)
print('The ROC-AUC score of the model is:', round(roc_auc_score(y_test, y_pred), 4))

The ROC-AUC score of the model is: 0.8621


### Scikit-Learn implementation of AdaBoost

In [9]:
ab_sk = AdaBoostClassifier(n_estimators = 400) # Same boosting rounds (M) as in our model
ab_sk.fit(X_train, y_train)
y_pred_sk = ab_sk.predict(X_test)
print('The ROC-AUC score of the model is:', round(roc_auc_score(y_test, y_pred_sk), 4))

The ROC-AUC score of the model is: 0.8427


Our custom model has a ROC-AUC score that's comparable to the Scikit-Learn implementation of AdaBoost. The Scikit-Learn implementation uses a different boosting algorithm, so our results are not exactly the same.

## Real dataset

Dataset containing 4601 emails that are labeled as either spam or not. Features are mostly word and symbol frequencies. The data is publicly available at the University of California, Irvine's Machine Learning repository and was donated by George Forman from HP Laboratories. It can be downloaded in this [link](http://archive.ics.uci.edu/ml/datasets/Spambase).

# Dataset
df = pd.read_csv('./spambase/spambase.data', header = None)

# Column names
names = pd.read_csv('./spambase/spambase.names', sep = ':', skiprows=range(0, 33), header = None)
col_names = list(names[0])
col_names.append('Spam')

# Rename df columns
df.columns = col_names

# Convert classes in target variable to {-1, 1}
df['Spam'] = df['Spam'] * 2 - 1

print('Shape of data:', df.shape)
df.tail()

In [27]:
# Train - test split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = 'Spam').values, df['Spam'].values, train_size = 0.8, random_state = 2)

In [15]:
### Custom implementation

Shape of data: (4601, 58)


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,Spam
4596,0.31,0.0,0.62,0.0,0.00,0.31,0.0,0.0,0.0,0.0,...,0.000,0.232,0.0,0.000,0.0,0.0,1.142,3,88,-1
4597,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.353,0.0,0.0,1.555,4,14,-1
4598,0.30,0.0,0.30,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.102,0.718,0.0,0.000,0.0,0.0,1.404,6,118,-1
4599,0.96,0.0,0.00,0.0,0.32,0.00,0.0,0.0,0.0,0.0,...,0.000,0.057,0.0,0.000,0.0,0.0,1.147,5,78,-1
4600,0.00,0.0,0.65,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.125,0.0,0.0,1.250,5,40,-1


In [28]:
# Fit model
ab = AdaBoost()
ab.fit(X_train, y_train, M = 400)

# Predict on test set
y_pred = ab.predict(X_test)
print('The ROC-AUC score of the model is:', round(roc_auc_score(y_test, y_pred), 4))

The ROC-AUC score of the model is: 0.9386


### Scikit-Learn implementation of AdaBoost

In [29]:
ab_sk = AdaBoostClassifier(n_estimators = 400) # Same boosting rounds (M) as in our model
ab_sk.fit(X_train, y_train)
y_pred_sk = ab_sk.predict(X_test)
print('The ROC-AUC score of the model is:', round(roc_auc_score(y_test, y_pred_sk), 4))

The ROC-AUC score of the model is: 0.9358
